In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
PPG_dict = {
    'PPG-CCMC': '55134',
    'PPG-Mat': '55135',
    'PPG-PROFMAT': '55136',
    'PPG-MECAI': '55137',
    'PPG-PIPGEs': '104131'
}

def cria_link( codigo, curso ):

    # codigo = codigo.strip()
    curso = curso.strip()

    try:
        ppg = PPG_dict[ curso ]
        return f'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa={ppg}&disciplina={codigo}'
    except KeyError:
        return f'https://uspdigital.usp.br/jupiterweb/obterDisciplina?nomdis=&sgldis={codigo}'


In [4]:
df = pd.read_excel('2026/1/Elenco_SME_2026-1.xlsx', skiprows = 2)
# df = pd.read_csv('../../Elenco SME_2025-2-ICMC.csv', delimiter=';')

In [5]:
df = df.dropna(subset=["Disciplina (código)"])
df = df.fillna( '' )

In [7]:
df = df.rename(columns={"Disciplina (código)": "codigo", 
                        "Disciplina (nome completo)": "nome", 
                        "Curso(s)": "curso",
                        "Horário 1": "horario1", "Horário 2": "horario2", "Turma": "turma"}, errors='ignore')
df = df.drop( [ 'Horário 3', 'Horário 4', 'observações', 'Docente\n(nome completo sem abreviações)', 'NUSP', 'Utilizará laboratório?\n(sim ou não)', 'Sala \n(a definir)', 'Será espelho com Pós?' , 'créditos' ], axis = 1, errors='ignore' )

In [114]:
ementa = []
nivel = []
for codigo, curso in zip( df[ 'codigo' ], df[ 'curso' ] ):
    ementa.append( cria_link( codigo, curso ) )
    try:
        p = PPG_dict[ curso ]
        nivel.append( 'p' )
    except KeyError:
        nivel.append( 'g' )

df[ 'ementa' ] = ementa
df[ 'nivel' ] = nivel

In [115]:
df

,codigo,nome,curso,turma,horario1,horario2,ementa,nivel
0,5500004,Fundamentos de Ciências de Dados,BCDados,1.0,Terça - 10:10/11:50,Quinta - 14:20/16:00,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
1,SME0104,Cálculo Numérico - vincular 5 vagas para o cur...,BCC,1.0,Terça - 08:10/09:50,Quinta - 10:10/11:50,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
2,SME0104,Cálculo Numérico,BCC,2.0,Terça - 10:10/11:50,Quinta - 08:10/09:50,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
3,SME0121,Processos Estocásticos,BCC,1.0,Terça - 10:10/11:50,Quinta - 08:10/09:50,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
4,SME0121,Processos Estocásticos,"BCC, BCDados",2.0,Terça - 08:10/09:50,Quinta - 10:10/11:50,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
...,...,...,...,...,...,...,...,...
78,SME5901,Otimização Linear I,CCMC,1.0,,,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
79,MAI5002,Fundamentos de Matemática Aplicada,MECAI,1.0,,,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
80,MAI5036,"Tópicos Especiais em Matematica, Estatística e...",MECAI,1.0,,,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
81,MAI5015,Visualização Científica,MECAI,1.0,,,https://uspdigital.usp.br/jupiterweb/obterDisc...,g


In [116]:
json_data = df.to_dict(orient='records')

In [117]:
for item in json_data:
    keys_to_remove = [k for k, v in item.items() if pd.isna(v) and k == "horario2"]
    for k in keys_to_remove:
        del item[k]

In [118]:
len(json_data)

83

In [119]:
dias_map = {
    "Segunda": "Seg.",
    "Terça": "Ter.",
    "Quarta": "Qua.",
    "Quinta": "Qui.",
    "Sexta": "Sex.",
    "Sábado": "Sáb.",
    "Domingo": "Dom."
}

In [120]:
def processar_horario(horario):
    if pd.isna(horario):
        return None
    try:
        # Exemplo: "Terça - 14:20 / 16:00"
        dia_e_horas = horario.split('-')
        dia_extenso = dia_e_horas[0].strip()
        dia = dias_map.get(dia_extenso, dia_extenso)  # Se não tiver no mapa, mantém original
        horas = dia_e_horas[1].strip().split('/')
        inicio = horas[0].strip()
        fim = horas[1].strip()
        return {"dia": dia, "inicio": inicio, "fim": fim}
    except Exception as e:
        print(f"Erro ao processar: {horario} -> {e}")
        return None

In [121]:
for item in json_data:
    horarios = []
    for key in ['horario1', 'horario2']:
        if key in item:
            h = processar_horario(item[key])
            if h:
                horarios.append(h)
            del item[key]  # Remove os campos antigos

    # if horarios:
    item['horarios'] = horarios

    if(not (isinstance(item['turma'], float) or isinstance(item['turma'], int))):
        item['turma'] = 1.0
        
    item['id'] = f"{item['codigo']},{int(item['turma'])}"
    item['codigo'] = str(item['codigo'])


Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao proce

In [122]:
def hora_para_minutos(hora_str):
    """Converte 'HH:MM' para minutos desde 00:00"""
    h, m = map(int, hora_str.split(":"))
    return h * 60 + m

In [123]:
def horarios_conflitam(h1, h2):
    """Verifica se dois horários conflitam no mesmo dia"""
    if h1['dia'] != h2['dia']:
        return False

    inicio1 = hora_para_minutos(h1['inicio'])
    fim1 = hora_para_minutos(h1['fim'])
    inicio2 = hora_para_minutos(h2['inicio'])
    fim2 = hora_para_minutos(h2['fim'])

    return max(inicio1, inicio2) < min(fim1, fim2)

In [124]:
# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "14:00", "fim": "16:01", "dia": "Seg."}

# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "18:00", "fim": "19:01", "dia": "Seg."}

horarios_conflitam(a, b)

False

In [125]:
# Inicializa os sets de conflitos
for turma in json_data:
    turma['conflitos'] = set()

# Compara todas as turmas em pares (sem repetições)
for i in range(len(json_data)):
    turma_i = json_data[i]
    for j in range(i + 1, len(json_data)):
        turma_j = json_data[j]

        # Verifica se algum horário da turma_i conflita com algum da turma_j
        for h1 in turma_i.get('horarios', []):
            for h2 in turma_j.get('horarios', []):
                if horarios_conflitam(h1, h2):
                    turma_i['conflitos'].add(turma_j['id'])
                    turma_j['conflitos'].add(turma_i['id'])
                    break  # Um conflito já basta

# Converte os sets para listas (JSON serializável)
for turma in json_data:
    turma['conflitos'] = list(turma['conflitos'])

In [126]:
def marcar_turmas_noturnas(json_data):
    for turma in json_data:
        turma['noturna'] = False  # valor padrão
        for horario in turma.get('horarios', []):
            try:
                hora, minuto = map(int, horario['inicio'].split(':'))
                if hora >= 18:
                    turma['noturna'] = True
                    break  # Já é noturna, não precisa verificar os outros horários
            except Exception as e:
                print(f"Erro ao processar horário {horario['inicio']} da turma {turma.get('id')}: {e}")


In [127]:
marcar_turmas_noturnas(json_data)

* Buscar nas páginas do ICMC da Graduação e da Pós para identificar qual o nível da disciplina, preencher a ementa com o link correto e buscar a quantidade de créditos

```
Pós - f'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa={ppg}&disciplina={codigo}'
Graduação - f'https://uspdigital.usp.br/jupiterweb/obterDisciplina?nomdis=&sgldis={codigo}'
```

In [128]:
import time

In [129]:
# --- INÍCIO DO SCRIPT ---

try:
    driver = webdriver.Chrome()
except Exception as e:
    print(f"Erro ao inicializar o WebDriver: {e}")
    print("Verifique se o Google Chrome está instalado.")
    exit()

dados_coletados_web = {}

print("Iniciando a busca por disciplinas...")

iteracao = 0
for turma in json_data:
    codigo = turma['codigo']
    disciplina_encontrada = False
    print(f"\n🔎 Processando disciplina: {codigo}")

    # 1. TENTATIVA NA GRADUAÇÃO
    url_graduacao = f'https://uspdigital.usp.br/jupiterweb/obterDisciplina?nomdis=&sgldis={codigo}'
    driver.get(url_graduacao)
    time.sleep(1) # Pequena pausa para garantir o carregamento da página

    try:
        # Verifica se a div de erro existe
        driver.find_element(By.ID, "web_mensagem")
        print(f"   - '{codigo}' não encontrada na Graduação. Verificando na Pós-Graduação...")

    except NoSuchElementException:
        # Se a div de erro NÃO existe, a disciplina é da graduação.
        print(f"   ✓ '{codigo}' encontrada na Graduação!")
        try:
            # Extrai o nome da disciplina
            nome_completo_tag = driver.find_element(By.XPATH, "//span[contains(., 'Disciplina:')]/b")
            # O texto é "Disciplina: SME0104 - Cálculo Numérico", pegamos só o nome
            nome_disciplina = nome_completo_tag.text.split('-', 1)[1].strip()

            # Extrai os créditos-aula
            # A busca por XPath é mais robusta:
            # Encontre o <b> com o texto 'Créditos Aula:', suba para a célula <td>, e pegue a próxima célula <td>
            creditos_aula_tag = driver.find_element(By.XPATH, "//b[contains(text(), 'Créditos Aula:')]/ancestor::td/following-sibling::td")
            creditos_aula = creditos_aula_tag.text.strip()

            tipo_disciplina_tag = driver.find_element(By.XPATH, "//b[contains(text(), 'Tipo:')]/ancestor::td/following-sibling::td")
            tipo_disciplina = tipo_disciplina_tag.text.strip()

            dados_coletados_web[codigo] = {
                'codigo': codigo,
                'nome': nome_disciplina,
                'nivel': 'g',
                'creditos': int(creditos_aula),
                'ementa_link': url_graduacao,
                'tipo': tipo_disciplina
            }
            disciplina_encontrada = True

        except Exception as e:
            print(f"      -> Erro ao extrair dados da página da Graduação para '{codigo}': {e}")
            dados_coletados_web[codigo] = {
                'codigo': codigo,
                'status': 'Erro na Extração (Graduação)',
                'url': url_graduacao
            }

    # 2. TENTATIVA NA PÓS-GRADUAÇÃO (se não foi encontrada na graduação)
    if not disciplina_encontrada:
        for ppg in PPG_dict.values():
            url_pos = f'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa={ppg}&disciplina={codigo}'
            driver.get(url_pos)
            time.sleep(1)

            try:
                # Seletor ATUALIZADO para o nome da disciplina na Pós
                nome_disciplina_pos = driver.find_element(By.CSS_SELECTOR, 'div.page-header h3').text.strip()
                
                # Seletor ATUALIZADO para o número de créditos na Pós
                creditos_p_tag = driver.find_element(By.XPATH, "//p[b[contains(text(), 'Nº de créditos:')]]")
                # Extrai o texto "Nº de créditos: 12", divide no ":" e pega a segunda parte
                creditos_pos = creditos_p_tag.text.split(':')[1].strip()

                duracao_element = driver.find_element(By.XPATH, "//tbody/tr[2]/td[count(//tbody/tr[1]/td[contains(., 'Duração')]/preceding-sibling::td) + 1]")
                
                duracao_valor = duracao_element.text

                print(f"   ✓ '{codigo}' encontrada na Pós-Graduação (Programa: {ppg})!")
                dados_coletados_web[codigo] = {
                    'codigo': codigo,
                    'nome': nome_disciplina_pos,
                    'nivel': 'p',
                    'creditos': int(creditos_pos),
                    'ementa_link': url_pos,
                    'tipo': "Semestral" if int(duracao_valor.split(" ")[0]) > 8 else "Bimestral" 
                }
                disciplina_encontrada = True
                break # Para o loop dos PPGs, pois já encontramos a disciplina
            except NoSuchElementException:
                print(f"   - '{codigo}' não encontrada no programa de Pós '{ppg}'.")
                continue

    # 3. Se não foi encontrada em nenhum lugar
    if not disciplina_encontrada:
        print(f"   ❌ '{codigo}' não encontrada em nenhuma das fontes.")
        dados_coletados_web[codigo] = {
            'codigo': codigo,
            'status': 'Não encontrada'
        }
    else: 
        print("Iteração: ", iteracao)

    iteracao+=1

# Fecha o navegador
driver.quit()

Iniciando a busca por disciplinas...

🔎 Processando disciplina: 5500004
   ✓ '5500004' encontrada na Graduação!
Iteração:  0

🔎 Processando disciplina: SME0104
   ✓ 'SME0104' encontrada na Graduação!
Iteração:  1

🔎 Processando disciplina: SME0104
   ✓ 'SME0104' encontrada na Graduação!
Iteração:  2

🔎 Processando disciplina: SME0121
   ✓ 'SME0121' encontrada na Graduação!
Iteração:  3

🔎 Processando disciplina: SME0121
   ✓ 'SME0121' encontrada na Graduação!
Iteração:  4

🔎 Processando disciplina: SME0130
   ✓ 'SME0130' encontrada na Graduação!
Iteração:  5

🔎 Processando disciplina: SME0202
   ✓ 'SME0202' encontrada na Graduação!
Iteração:  6

🔎 Processando disciplina: SME0205
   ✓ 'SME0205' encontrada na Graduação!
Iteração:  7

🔎 Processando disciplina: SME0221
   ✓ 'SME0221' encontrada na Graduação!
Iteração:  8

🔎 Processando disciplina: SME0230
   ✓ 'SME0230' encontrada na Graduação!
Iteração:  9

🔎 Processando disciplina: SME0243
   ✓ 'SME0243' encontrada na Graduação!
Iteração

In [130]:
TCC_ESTAGIO = (
    "Trabalho de Conclusão de Curso",
    "Estágio Supervisionado"
)

In [131]:
for turma in json_data:
    dados = dados_coletados_web[turma['codigo']]
    turma['codigo'] = turma['codigo']
    turma['nome'] = dados['nome']
    turma['nivel'] = dados['nivel']
    # turma['créditos'] = int(dados['creditos'])
    turma['ementa'] = dados['ementa_link']

    turma['turma'] = 1 if not turma['turma'] else int(turma['turma'])

    # Calcular a carga didática

    # Graduação
    if(turma['nivel'] == 'g'):
        if any(termo in turma['nome'] for termo in TCC_ESTAGIO):
            turma['carga'] = 0.25
        else:
            turma['carga'] = 0.25 * int(dados['creditos'])

    # Pós
    elif(turma['nivel'] == 'p'):
        if(dados['creditos'] >= 3): 
            if(dados['tipo'] == "Semestral" ):
                turma['carga'] = 1.0
            else:
                turma['carga'] = 0.5
        else:
             turma['carga'] = 0
        



In [132]:
json_data.sort(key=lambda x: x['codigo'])

In [133]:
import json

In [134]:
with open(r'public/turmas.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [135]:
with open(r'./app/context/dados.ts', 'w', encoding='utf-8') as f:
    f.write('import { TurmaDataInicial } from "../types";\n')
    f.write('\nexport const turmasJson: TurmaDataInicial[] = ')
    json.dump(json_data, f, ensure_ascii=False, indent=2)